In [1]:
# Import necessary libraries
import pandas as pd

# Load the dataset from the CSV file
data = pd.read_csv("SPAM text message 20170820 - Data.csv")

# Display the first few rows of the dataset to understand its structure
data.head()


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [2]:
# Import necessary libraries
import string

# Function to preprocess text
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation and special characters
    text = ''.join([char for char in text if char not in string.punctuation])
    return text

# Apply the preprocessing function to the 'Message' column
data['Message'] = data['Message'].apply(preprocess_text)

# Display the first few rows of the preprocessed dataset
data.head()


,Category,Message
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i dont think he goes to usf he lives aroun...


In [3]:
# Import necessary libraries
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize and fit the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Message'])

# Convert text messages to sequences of integers
sequences = tokenizer.texts_to_sequences(data['Message'])

# Pad the sequences to have the same length
padded_sequences = pad_sequences(sequences, padding='post')

# Display the first few rows of the tokenized and padded sequences
padded_sequences[:5]


array([[  46,  441, 4382,  794,  713,  679,   64,    9, 1250,   90,  120,
         354, 1251,  153, 2905, 1252,   68,   57, 4383,  137,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [4]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

# Build the RNN model
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
embedding_dim = 16
max_length = len(max(padded_sequences, key=len))

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    SimpleRNN(32),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary (architecture)
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 171, 16)           154528    
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                1568      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 156,129
Trainable params: 156,129
Non-trainable params: 0
_________________________________________________________________


In [5]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
import numpy as np

# Convert the labels to integers (0 or 1)
labels = np.array(data['Category'].map({'ham': 0, 'spam': 1}))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)


In [6]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/10
112/112 [==============================] - 8s 55ms/step - loss: 0.3265 - accuracy: 0.8954 - val_loss: 0.1634 - val_accuracy: 0.9563
Epoch 2/10
112/112 [==============================] - 5s 41ms/step - loss: 0.1360 - accuracy: 0.9683 - val_loss: 0.1379 - val_accuracy: 0.9608
Epoch 3/10
112/112 [==============================] - 6s 55ms/step - loss: 0.0666 - accuracy: 0.9849 - val_loss: 0.1282 - val_accuracy: 0.9641
Epoch 4/10
112/112 [==============================] - 5s 45ms/step - loss: 0.2327 - accuracy: 0.9136 - val_loss: 0.0995 - val_accuracy: 0.9742
Epoch 5/10
112/112 [==============================] - 5s 46ms/step - loss: 0.0413 - accuracy: 0.9927 - val_loss: 0.1092 - val_accuracy: 0.9720
Epoch 6/10
112/112 [==============================] - 5s 49ms/step - loss: 0.0306 - accuracy: 0.9952 - val_loss: 0.0978 - val_accuracy: 0.9742
Epoch 7/10
112/112 [==============================] - 5s 41ms/step - loss: 0.0282 - accuracy: 0.9950 - val_loss: 0.1141 - val_accuracy: 0.9709

In [7]:
# Evaluate the model on the testing data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

# Print the test accuracy
print("\nTest Accuracy: {:.4f}".format(test_accuracy))


35/35 [==============================] - 1s 30ms/step - loss: 0.2534 - accuracy: 0.9336

Test Accuracy: 0.9336
